# Data Engineering Activity
This demonstration is to teach data engineering principles that can be applied to a wide range of datasets and data types. In this case, we are working with a synthetic dataset based on drill data.

## Setup

Run these cells to load in the approprate packages and make sure the notebook has the data accessible. You should run this on a Jupyter Notebook downloaded earlier in the training.

In [ ]:
# Uncomment if you want full environment setup
# !pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from pandas_profiling import ProfileReport
import pickle
# import tensorflow as tf
warnings.filterwarnings("ignore")
try:
    from rop_utils import ROPData
    data_folder = "../data/"
except ModuleNotFoundError:
    !git clone https://github.com/pvankatwyk/vermeer-training.git
    data_folder = "vermeer-training/data/"

%matplotlib inline

## Step 1: Load Data

The first step is to load the data into Python. The data is currently stored in a Comma Separated Values format (CSV) on your computer's hard drive. In order for python to be able to interface with the data, it will load the data into RAM for easy access and calculation.  

Below are a few different file types that python can handle using the Pandas library. For more information on pandas, see the documentation here:  . The same data is loaded from a CSV file, a TXT file, and a XLSX file. Do they look the same?

In [ ]:
# Load data from .csv, .txt, and .xlsx files
# TODO: Load in the data from both the text file and the csv file (optional, xlsx file also)

# Are they the same?
# TODO: Write some code to make sure they are the same

In [ ]:
# Load in data for this training
data = pd.read_csv(data_folder + r'synthetic_drill_data_1.csv')
data

## Step 2: Calculating ROP

The goal for this activity is to learn about and be able to predict the speed of drilling, or Rate of Penetration (ROP). ROP is not a measurement found in the existing data, so we must calculate it and add it to the dataset. We will look at two rows at a time and take the difference in drill depth divided by the difference in timestamps. This will give us a measurement in distance per time (speed), or in our case, feet per minute.

In [ ]:
def process(data):
    # Convert to pandas-recognized timestamp
    data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])
    n = len(data)
    # Create an array for storing the data - 
    deltaTime = np.zeros(n) * np.nan  # differences in timestamps
    forward = [True] * n              # whether the drill is moving forward

    # Calculate time stamp differences
    for i in range(1, n):
        deltaTime[i] = (data['TimeStamp'][i] - data['TimeStamp'][i - 1]).total_seconds()
        forward[i] = True if data['RodCount'][i] > data['RodCount'][i - 1] else False

    # average of 2 and 3 for 1st time point only
    deltaTime[0] = np.mean(deltaTime[1:3])
    
    # calculate ROP from deltaTime and 10 feet difference between rods
    data['ROP (ft/min)'] = (60 * 10.0 / deltaTime)
    
    # Only include data when the drill is moving forward
    data = data[forward]
    
    # Drop rows with no time change
    data = data.replace([np.inf, -np.inf], np.nan)
    data = data.reset_index(drop=True)
    
    return data

def filter(data, column, min=None, max=None):
    if max is None:
        max = data[column].max()
    if min is None:
        min = data[column].min()

    return data[(data[column] < max) & (data[column] > min)]

In [ ]:
# Process data
# TODO: process the data

# Filter the data -- ROP > 0
# TODO: filter the data
#  - Use filter() above
filtered_data = 

data = filtered_data.drop(columns=['Id', 'TimeStamp', 'Latitude', 'Longitude', "Thrust Speed Avg (ft/min)"])

In [ ]:
data

In [ ]:
data.columns

In [ ]:
# You can also do this to get the same output...
# data = ROPData().upload(data_folder + 'synthetic_drill_data.csv').process().filter(rop_greater_than=0)

## Step 3: Removing Bad Data

### Identifying Missing Data  
First we need to identify if our dataset has missing fields. The following code sums the number of values that are missing in each column.

In [ ]:
# TODO: Sum up all the na values in the columns
#  - hint: .isna()

### Handling Missing Data  
There are a few different ways of handling missing fields within the data. The best way to deal with missing data is to simply delete the entire row where the field is missing. Since the field is empty, there is no way to know what should go there and we ideally don't want to impose prior knowledge into the data.  

However, sometimes the data is not long enough and we cannot afford to delete any of the rows. In that case, you can use imputation, which is a way of filling in the missing field with a logical "best guess" such as the column mean, median, or an interpolation between the row before and after. In this exercise, we will simply delete rows with missing data.

###### Deleting Rows with Missing Information

In [ ]:
# TODO: Drop NA values

In [ ]:
# TODO: fill in the same code you wrote above (summing up na values)

###### Imputing and Interpolating Missing Information

Imputation Example (from [Sklearn Docs](https://scikit-learn.org/stable/modules/impute.html)):

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit([[1, 2], [np.nan, 3], [7, 6]])
SimpleImputer()
X = [[np.nan, 2], [6, np.nan], [7, 6]]
print(imp.transform(X))

Interpolation Example (from [SciPy Docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d)):

In [ ]:
from scipy import interpolate
x = np.arange(0, 10)
y = np.exp(-x/3.0)
f = interpolate.interp1d(x, y)

xnew = np.arange(0, 9, 0.1)
ynew = f(xnew)   # use interpolation function returned by `interp1d`
plt.plot(x, y, 'o', xnew, ynew, '-')
plt.show()

### Identifying Outliers  
Now that the data is loaded and missing data is deleted, we need to identify rows that likely are not representative of the true data that we are trying to learn about. This may include faulty measurements due to sensor error, operator error, or simply drillruns that are abnormal. These rows are called outliers.

Histograms and boxplots are great ways of looking for outliers. Histograms plot the frequency (count) of data points within a given measurement range. For example, we may see 10 rows that have ROP within 3-4 ft/min. Boxplots are particularly helpful in identifying outliers by using quartiles ranges. For more information on identifying outliers from plots, see this article:  .

In [ ]:
def histograms(columns):
    for column in columns:
        sns.histplot(data[column])
        plt.title(f'Histogram for {column}')
        plt.show()
        print('')

def boxplots(columns):
    for column in columns:
        sns.boxplot(data[column])
        plt.title(f'Boxplot for {column}')
        plt.show()
        
        # Calculate quartile ranges
        Q1 = data[column].quantile(0.25)
        Q2 = data[column].quantile(0.5)
        Q3 = data[column].quantile(0.75)

        # Calculate Inter-Quartile Range (IQR)
        IQR = Q3 - Q1
        max_ = Q3 + 1.5*IQR
        min_ = Q1 - 1.5*IQR
        
        min_ = min_ if min_ > 0 else 0

        print(f"{column} -- Median (Q2): {Q2}, Max: {max_}, Min: {min_}")
        print('')

columns = ['Rotation Speed Max (rpm)', 'Thrust Force Max (lbf)', 'Drill String Length (ft)', 'ROP (ft/min)']
boxplots(columns)
histograms(columns)

### Removing Outliers
As you can see from the plots above, there are many outliers in the data that need to be removed. Removing outliers can be very subjective in approach but mathematical principles can be employed to keep the process uniform. We will use the bounds of the boxplots as good boundaries for outlier regions. Below are two functions to help you calculate the data. A typical value for the outlier range is 1.5 (as seen in the boxplot), but we use a value of 2.5 to be inclusive of rows that are extreme but may not be outliers.

In [ ]:
def calculate_bounds(data, outlier_range=1.5):
    bounds = {}
    for column in data.columns:
        try:
            Q1 = data[column].quantile(0.25)
            Q2 = data[column].quantile(0.5)
            Q3 = data[column].quantile(0.75)

            IQR = Q3 - Q1
            max_ = Q3 + outlier_range*IQR
            min_ = Q1 - outlier_range*IQR
            
            min_ = min_ if min_ > 0 else 0

            bounds[column] = {'min_': min_, 'max_': max_}
        except TypeError:
            pass
    return bounds

def delete_outliers(data, outlier_range=1.5):
    bounds = calculate_bounds(data, outlier_range=outlier_range)
    for column in bounds.keys():
        data = data[(data[column] <= bounds[column]['max_']) & (data[column] >= bounds[column]['min_'])]
    return data

In [ ]:
data = delete_outliers(data, outlier_range=2.5)

In [ ]:
boxplots(columns)
histograms(columns)

The data now looks much more representative of real values and is ready to be visualized and modeled.

## Step 4: Visualizing Data

Visualization is an important step in understanding the underlying principles governing your data. There are hundreds of different kinds of plots we can use to plot data. Below are a few different functions that will help you plot the data. Try using the functions to plot different columns in the dataset and start thinking about what patterns you see may tell us about the underlying processes.

#### Plotting Functions

In [ ]:
def scatterplot(data, x_column, y_column, fits=None):
    x = data[x_column]
    y = data[y_column]

    plt.scatter(x, y)
    plt.title(f'{x_column} vs {y_column}')
    plt.xlabel(str(x_column))
    plt.ylabel(str(y_column))

    if fits is None:
        return None
    
    elif "linear" in fits.lower():
        fit = np.polyfit(x,y,1, full=False)
        x_plot = np.linspace(min(x), max(x), 1000)
        plt.plot(x_plot, np.polyval(fit, x_plot), color='r', label=f'Linear Fit ($x$)')
        plt.legend()
        interpretation = f'As {column} increases by 1 unit, ROP changes by {round(fit[0],6)} ft/min'
    elif 'quadratic' in fits.lower() or 'square' in fits.lower():
        fit = np.polyfit(x,y,2, full=False)
        plt.plot(x_plot, np.polyval(fit, x_plot), color='green', label=f'Quadratic Fit ($x^2$)')
        plt.legend()
        interpretation = None
    elif 'cub' in fits.lower():
        fit = np.polyfit(x,y,3, full=False)
        plt.plot(x_plot, np.polyval(fit, x_plot), color='orange', label=f'Cubic Fit ($x^3$)')
        plt.legend()
        interpretation = None
    else:
        fit = None
        interpretation = None
    return fit, interpretation

def density(data, x_column, y_column, bins=(20,20), cmin=None, cmax=None):
    x = data[x_column]
    y = data[y_column]
    plt.hist2d(x, y, bins=(20,20), cmap=plt.cm.jet, cmin=cmin, cmax=cmax)
    plt.xlabel(str(x_column))
    plt.ylabel(str(y_column))
    plt.title(f'Density Plot of {y_column} vs {x_column}')
    plt.colorbar()
    return None


def plot_all(data, column):
    for column in columns:
        x = column
        y = 'ROP (ft/min)'
        plt.figure(figsize=(15,5))
        plt.subplot(1,2,1)

        # Scatterplots
        plt.scatter(data[x],data[y])
        plt.title(f'{x} vs {y}')
        plt.xlabel(str(x))
        plt.ylabel(str(y))
        # plt.show()

        plt.subplot(1,2,2)     
        plt.hist2d(data[x], data[y], bins=(20,20), cmap=plt.cm.jet)
        plt.title(f'{x} vs {y}')
        plt.xlabel(str(x))
        plt.ylabel(str(y))
        plt.show()

        print('')

### Make Plots

In [ ]:
# Print out the available columns to plot
data.columns

In [ ]:
sample = data.sample(1000)
col = 'Rotation Speed Max (rpm)' # Change this value to plot different variables
scatterplot(sample, col, 'ROP (ft/min)')

In [ ]:
density(sample, x_column=col, y_column='ROP (ft/min)', bins=(20,20), cmin=None, cmax=None)

In [ ]:
# Plot all columns (except for one at a time)
columns = ['RodCount','Rotation Speed Max (rpm)',
                             'Rotation Torque Max (ft-lb)', 'Thrust Force Max (lbf)', 
                             'Mud Flow Rate Avg (gpm)', 'Mud Pressure Max (psi)',
                             'Pull Force Maximum (lbf)', 'Pull Speed Average (ft/min)', 
                             'Drill String Length (ft)',]
plot_all(data, columns)

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(data.corr(), cmap='coolwarm', vmin=-1, vmax=1)

In [ ]:
# smaller_sample = data.sample(300)
# profile = ProfileReport(smaller_sample)
# try:
#    profile.to_notebook_iframe()         # view as widget in Notebook
# except:
#    profile.to_file('data.html') # save as html file

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data['Model'], data['ROP (ft/min)'], orient='v')

## Step 5: Extracting Insights

When we visualize the data we can make qualitative assessments of the data. However, qualitative observations often need to be backed by quantitative analysis. So how can we extract insights from our data in a concrete way?

The first way to is to use summary statistics. The next cell shows a simple pandas function that gives summary statistics for the entire dataset. What do you notice? The second cell gives summary statistics for those rows whose ROP is in the top 25% (top quartile). How do they differ? Could you infer some of the reasons they may differ?

The second way is to fit functions to your data. In this example, you can use a linear fit and use the coefficient to understand the effect that a unit change in x can have on y. Try different variables to see which variables show stronger correlations with ROP.

In [ ]:
data.describe()

In [ ]:
# What is the data like when ROP is in the top 25%?
# TODO: Describe the data with rows where ROP is in the top 25%
#  - hint: column.quantile(0.75)
#  - hint: new_data = data[condition]

In [ ]:
# How does each model perform?
# TODO: Find out the mean values for each column GROUPED BY (hint) the Model type

# grouping by Id  --  we deleted this but could this be useful?

In [ ]:
# How does Drill String Length affect ROP?
column = 'Rotation Torque Max (ft-lb)'
fit, interpretation = scatterplot(data, column, 'ROP (ft/min)', fits='linear')
plt.show()
print(interpretation)

## Step 6: DrillGIS

Now we will look at an example of a dashboard, or a tool to view and analyze data that is generally hosted online. Click the link below and click around the website. The website shows fake drilling data that can be used to compare your drilling performance with those around you. You can also log into the website with code 571167 as a manager and 563624 as an operator.

[https://drillgis.com](https://drillgis.com)

## Step 7: Feature Engineering

The following cells show some things you can do to try and prepare the features in your dataset for a Machine Learning model. These include one-hot encoding, scaling, and other processing.

In [ ]:
encoder = OneHotEncoder(sparse=False)
encoded = encoder.fit_transform(data[['Model']])
for i, model in enumerate(encoder.categories_[0]):
    data[model] = encoded[:,i]

data = data.drop(columns=['Model'])

In [ ]:
data.head()

In [ ]:
columns = data.columns
scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data))
data.columns = columns

In [ ]:
data.head()

## Step 8: ML Data Processing
The next steps are necessary for the analysis of the ML algorithms. First, we identify which data columns are the features and target. Then we split the data according to a train (80%) and a test (20%) set.

In [ ]:
# TODO: Assign X to the feature columns (variables) and y to the target (what we're predicting)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(# TODO: Fill in)

## Step 9: ML Implementation
Now we will train and deploy a very simple ML model. How did it do? What other metrics could you use to measure your performance?

### Machine Learning

In [ ]:
model = ExtraTreesRegressor()
# TODO: Fit the model and make predictions on test dataset

In [ ]:
mae = mean_absolute_error(# TODO: Fill in)
mse = mean_squared_error(# TODO: Fill in)
print(f'Mean Absolute Error (xp - x): {round(mae,6)}')
print(f'Mean Squared Error (xp - x)^2: {round(mse,6)}')

### Deploying and Using a ML Model

In [ ]:
# TODO: Use pickle package to dump the model to a file

In [ ]:
# TODO: Load the file that you just dumped (loads in saved model)

In [ ]:
# TODO: Make sure the models are the same. How would you do that?